# LangChain Introduction

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

### Prompts:

1. PromptTemplate qui permet de créer des prompts avec des variables.

In [2]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Raconte-moi une blague sur {topic}")

prompt.format(topic="les développeurs")


/scratch/brikiyou/ift6289/IFT6285-fine-tuning-demo/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


'Raconte-moi une blague sur les développeurs'

2. ChatPromptTemplate qui permet de créer des prompts structurés avec des messages de différents types (système, utilisateur, assistant) pour les LLMs.

In [3]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "Tu es un assistant expert en {domaine}."),
    ("human", "Explique-moi le concept de {concept}."),
])

template.format_messages(domaine="finance", concept="dividendes")

[SystemMessage(content='Tu es un assistant expert en finance.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Explique-moi le concept de dividendes.', additional_kwargs={}, response_metadata={})]

1. FewShotPromptTemplate permet de fournir des exemples au modèle pour guider sa réponse (Few-Shot Learning). Cela améliore considérablement la précision pour des tâches complexes ou des formats spécifiques

In [6]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

examples = [
    {"input": "Heureux", "output": "Triste"},
    {"input": "Grand", "output": "Petit"},
]

example_prompt = PromptTemplate.from_template("Mot: {input}\nAntonyme: {output}")

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Mot: {input}\nAntonyme:", # Le nouveau mot à traiter
    input_variables=["input"]
)

prompt.format(input="Rapide")

'Mot: Heureux\nAntonyme: Triste\n\nMot: Grand\nAntonyme: Petit\n\nMot: Rapide\nAntonyme:'

### Models: